In [1]:
%load_ext autoreload
%autoreload 2

import selenium.webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement

driver = selenium.webdriver.Chrome()

In [2]:
from property_models.aus.old_listings.constants import OldListingURL

old_listing_url = OldListingURL(
    state="VIC",
    suburb="Ascot Vale",
    postcode=3032,
    page=1,
    beds=2,
    baths=2,
    cars=2,
)
old_listing_url

OldListingURL(state='VIC', suburb='Ascot Vale', postcode=3032, page=1, beds=2, baths=2, cars=2)

In [3]:
page = driver.get(old_listing_url.format())

In [4]:
postcode = old_listing_url.postcode
state = old_listing_url.state

In [5]:
page_navigation = driver.find_element(By.CLASS_NAME, "pagination")
last_navigation = page_navigation.find_elements(By.TAG_NAME, "li")[-2]
int(last_navigation.text)

2

In [6]:
site = "https://www.oldlistings.com.au/real-estate/{state}/{suburb}/{postcode}/buy/{page}/:bed:{beds}:bedmax:{beds}:bath:{baths}:car:{cars}"
site = "https://www.oldlistings.com.au/real-estate/VIC/ascot vale/3032/buy/3/:bed:2:bedmax:2:bath:1:car:1:cat:Apartment"
driver.get(site)

In [7]:
table = driver.find_elements(By.CLASS_NAME, "content-col")[0]
table

<selenium.webdriver.remote.webelement.WebElement (session="25a1fa8933097c5845250873a0087379", element="f.3E412B7AAFF8C20FDF066546276C4DC5.d.DC4A0F490046C766D2AD69357AF32559.e.141")>

In [8]:
listings = table.find_elements(By.TAG_NAME, "div")
listings

[<selenium.webdriver.remote.webelement.WebElement (session="25a1fa8933097c5845250873a0087379", element="f.3E412B7AAFF8C20FDF066546276C4DC5.d.DC4A0F490046C766D2AD69357AF32559.e.132")>,
 <selenium.webdriver.remote.webelement.WebElement (session="25a1fa8933097c5845250873a0087379", element="f.3E412B7AAFF8C20FDF066546276C4DC5.d.DC4A0F490046C766D2AD69357AF32559.e.133")>,
 <selenium.webdriver.remote.webelement.WebElement (session="25a1fa8933097c5845250873a0087379", element="f.3E412B7AAFF8C20FDF066546276C4DC5.d.DC4A0F490046C766D2AD69357AF32559.e.134")>,
 <selenium.webdriver.remote.webelement.WebElement (session="25a1fa8933097c5845250873a0087379", element="f.3E412B7AAFF8C20FDF066546276C4DC5.d.DC4A0F490046C766D2AD69357AF32559.e.135")>,
 <selenium.webdriver.remote.webelement.WebElement (session="25a1fa8933097c5845250873a0087379", element="f.3E412B7AAFF8C20FDF066546276C4DC5.d.DC4A0F490046C766D2AD69357AF32559.e.136")>,
 <selenium.webdriver.remote.webelement.WebElement (session="25a1fa8933097c584525

In [9]:
from property_models.aus.old_listings.extract import extract_listing_data

In [10]:
listing = listings[0]
raw_listing = extract_listing_data(listing, state=state, postcode=postcode)
raw_listing

RawListing(general_info=RawPropertyInfo(address='2/68 ORMOND ROAD, ASCOT VALE, VIC 3032', beds='2', cars='1', baths='1', property_type='Unit/apmt'), recent_price=RawPriceRecord(date=' March 2019', market_info='$380,000'), historical_prices=[RawPriceRecord(date='March 2019', market_info='$380,000 - $415,000'), RawPriceRecord(date='June 2013', market_info='Auction'), RawPriceRecord(date='June 2013', market_info='$315,000 - $345,000'), RawPriceRecord(date='June 2013', market_info='$315,000 - $345,000 Auction'), RawPriceRecord(date='August 2011', market_info='Private Sale')])

In [17]:
raw_listing.to_property_info()

PropertyInfo(address=Address(unit_number=2, street_number=68, street_name='ORMOND ROAD', suburb='ASCOT VALE', postcode=3032, state='VIC', country='AUS'), beds=2, baths=1, cars=1, property_size_m2=None, land_size_m2=None, condition=None, property_type=None, construction_date=None, floors=None)

In [18]:
raw_listing.recent_price.date

' March 2019'

In [19]:
raw_listing.recent_price.date

' March 2019'

In [20]:
from property_models.aus.old_listings.parse import parse_date

parse_date(raw_listing.recent_price.date)

datetime.date(2019, 3, 1)

In [21]:
from datetime import date

In [23]:
raw_listing.to_property_info()

PropertyInfo(address=Address(unit_number=2, street_number=68, street_name='ORMOND ROAD', suburb='ASCOT VALE', postcode=3032, state='VIC', country='AUS'), beds=2, baths=1, cars=1, property_size_m2=None, land_size_m2=None, condition=None, property_type=None, construction_date=None, floors=None)

In [24]:
raw_listing.to_price_records()

[PriceRecord(address=Address(unit_number=2, street_number=68, street_name='ORMOND ROAD', suburb='ASCOT VALE', postcode=3032, state='VIC', country='AUS'), date=datetime.date(2019, 3, 1), record_type=None, price=380000),
 PriceRecord(address=Address(unit_number=2, street_number=68, street_name='ORMOND ROAD', suburb='ASCOT VALE', postcode=3032, state='VIC', country='AUS'), date=datetime.date(2019, 3, 1), record_type=None, price=397500),
 PriceRecord(address=Address(unit_number=2, street_number=68, street_name='ORMOND ROAD', suburb='ASCOT VALE', postcode=3032, state='VIC', country='AUS'), date=datetime.date(2013, 6, 1), record_type=<RecordType.AUCTION: 'auction'>, price=None),
 PriceRecord(address=Address(unit_number=2, street_number=68, street_name='ORMOND ROAD', suburb='ASCOT VALE', postcode=3032, state='VIC', country='AUS'), date=datetime.date(2013, 6, 1), record_type=None, price=330000),
 PriceRecord(address=Address(unit_number=2, street_number=68, street_name='ORMOND ROAD', suburb='AS

In [25]:
### FIXME
extracted_data = []
for listing in listings:
    extracted_data.append(listing.process_property_info())
extracted_data

AttributeError: 'WebElement' object has no attribute 'process_property_info'

In [ ]:
historical_prices = [data["historical_prices"] for data in extracted_data if data is not None]
historical_prices

[[{'date': 'March 2019', 'price': 0, 'type': '$380,000 - $415,000'},
  {'date': 'June 2013', 'price': 0, 'type': 'Auction'},
  {'date': 'June 2013', 'price': 0, 'type': '$315,000 - $345,000'},
  {'date': 'June 2013', 'price': 0, 'type': '$315,000 - $345,000 Auction'},
  {'date': 'August 2011', 'price': 0, 'type': 'Private Sale'}],
 [{'date': 'January 2019', 'price': 0, 'type': '$495,000'}],
 [{'date': 'October 2018', 'price': 0, 'type': '$440 Private Sale'}],
 [{'date': 'September 2018', 'price': 0, 'type': '$430,000 Private Sale'}],
 [{'date': 'July 2018', 'price': 0, 'type': 'By Negotiation'},
  {'date': 'October 2014', 'price': 0, 'type': '$320,000 - $350,000 Auction'}],
 [{'date': 'June 2018', 'price': 0, 'type': '$492,000'},
  {'date': 'June 2018', 'price': 0, 'type': 'Auction'},
  {'date': 'June 2018', 'price': 0, 'type': '$440,000 - $480,000'},
  {'date': 'February 2015', 'price': 0, 'type': '$455,000'},
  {'date': 'July 2012', 'price': 0, 'type': '$410,000 Price Guide'},
  {'da

In [ ]:
list_of_historical_prices = [historical_price["type"] for historical_price in sum(historical_prices, [])]
list_of_historical_prices

['$380,000 - $415,000',
 'Auction',
 '$315,000 - $345,000',
 '$315,000 - $345,000 Auction',
 'Private Sale',
 '$495,000',
 '$440 Private Sale',
 '$430,000 Private Sale',
 'By Negotiation',
 '$320,000 - $350,000 Auction',
 '$492,000',
 'Auction',
 '$440,000 - $480,000',
 '$455,000',
 '$410,000 Price Guide',
 '$415,000',
 '$415,000',
 '$350 Week',
 '$360,000 - $395,000',
 'Private Sale',
 '$400,000 - $440,000',
 '$400,000 - $440,000 Auction',
 'Auction',
 'Contact',
 'Auction',
 '$480,000 - $520,000',
 '$480,000 - $520,000 Auction',
 'Contact',
 '$625,000',
 '$649,950',
 '$649,950',
 'Contact',
 '$670,000',
 '$625,000',
 '$490,000 Private Sale',
 '$575,000',
 '$585,000',
 '$702,000',
 '$460,000 - $500,000 Auction',
 '$460,000 - $500,000',
 'Auction',
 'Contact',
 '$540,000 - $580,000',
 'Auction',
 '$480,000',
 '$465,000 Private Sale',
 '$440,000 - $480,000',
 '$300,000 In excess of',
 '$450,000 - $490,000 Auction',
 'Auction',
 '$450,000 - $490,000',
 '$410,000 - $450,000',
 'Private Sa

In [ ]:
import re


def convert_case(match_obj):
    if match_obj.group(1) is not None:
        return match_obj.group(1).lower()
    if match_obj.group(2) is not None:
        return match_obj.group(2).upper()


str = "jOE kIM mAx ABY lIzA"
print(re.sub(r"([A-Z]+)|([a-z]+)", convert_case, str))

Joe Kim MaX aby LiZa


In [ ]:
from property_models.aus.old_listings.process import parse_price

for price_info in list_of_historical_prices:
    price = parse_price(price_info)

    print(price_info, "=>", price)

$380,000 - $415,000 => 397500.0
Auction => None
$315,000 - $345,000 => 330000.0
$315,000 - $345,000 Auction => 330000.0
Private Sale => None
$495,000 => 495000.0
$440 Private Sale => 440.0
$430,000 Private Sale => 430000.0
By Negotiation => None
$320,000 - $350,000 Auction => 335000.0
$492,000 => 492000.0
Auction => None
$440,000 - $480,000 => 460000.0
$455,000 => 455000.0
$410,000 Price Guide => 410000.0
$415,000 => 415000.0
$415,000 => 415000.0
$350 Week => 350.0
$360,000 - $395,000 => 377500.0
Private Sale => None
$400,000 - $440,000 => 420000.0
$400,000 - $440,000 Auction => 420000.0
Auction => None
Contact => None
Auction => None
$480,000 - $520,000 => 500000.0
$480,000 - $520,000 Auction => 500000.0
Contact => None
$625,000 => 625000.0
$649,950 => 649950.0
$649,950 => 649950.0
Contact => None
$670,000 => 670000.0
$625,000 => 625000.0
$490,000 Private Sale => 490000.0
$575,000 => 575000.0
$585,000 => 585000.0
$702,000 => 702000.0
$460,000 - $500,000 Auction => 480000.0
$460,000 - 

In [ ]:
from property_models.aus.old_listings.process import parse_record_type
from property_models.constants import RecordType

for price_info in list_of_historical_prices:
    type = parse_record_type(price_info)

    print(price_info, " : ", type)

$380,000 - $415,000  :  None
Auction  :  RecordType.AUCTION
$315,000 - $345,000  :  None
$315,000 - $345,000 Auction  :  RecordType.AUCTION
Private Sale  :  RecordType.PRIVATE_SALE
$495,000  :  None
$440 Private Sale  :  RecordType.PRIVATE_SALE
$430,000 Private Sale  :  RecordType.PRIVATE_SALE
By Negotiation  :  RecordType.PRIVATE_SALE
$320,000 - $350,000 Auction  :  RecordType.AUCTION
$492,000  :  None
Auction  :  RecordType.AUCTION
$440,000 - $480,000  :  None
$455,000  :  None
$410,000 Price Guide  :  RecordType.ENQUIRY
$415,000  :  None
$415,000  :  None
$350 Week  :  RecordType.RENT
$360,000 - $395,000  :  None
Private Sale  :  RecordType.PRIVATE_SALE
$400,000 - $440,000  :  None
$400,000 - $440,000 Auction  :  RecordType.AUCTION
Auction  :  RecordType.AUCTION
Contact  :  RecordType.ENQUIRY
Auction  :  RecordType.AUCTION
$480,000 - $520,000  :  None
$480,000 - $520,000 Auction  :  RecordType.AUCTION
Contact  :  RecordType.ENQUIRY
$625,000  :  None
$649,950  :  None
$649,950  :  No

In [ ]:
def clean_market_info(price_info) -> tuple[float | None, RecordType | None]:
    record_type = parse_record_type(price_info)
    price = parse_price(price_info)

    return price, record_type


for price_info in list_of_historical_prices:
    print(price_info, " : ", clean_market_info(price_info))

$380,000 - $415,000  :  (397500.0, None)
Auction  :  (None, <RecordType.AUCTION: 'auction'>)
$315,000 - $345,000  :  (330000.0, None)
$315,000 - $345,000 Auction  :  (330000.0, <RecordType.AUCTION: 'auction'>)
Private Sale  :  (None, <RecordType.PRIVATE_SALE: 'private_sale'>)
$495,000  :  (495000.0, None)
$440 Private Sale  :  (440.0, <RecordType.PRIVATE_SALE: 'private_sale'>)
$430,000 Private Sale  :  (430000.0, <RecordType.PRIVATE_SALE: 'private_sale'>)
By Negotiation  :  (None, <RecordType.PRIVATE_SALE: 'private_sale'>)
$320,000 - $350,000 Auction  :  (335000.0, <RecordType.AUCTION: 'auction'>)
$492,000  :  (492000.0, None)
Auction  :  (None, <RecordType.AUCTION: 'auction'>)
$440,000 - $480,000  :  (460000.0, None)
$455,000  :  (455000.0, None)
$410,000 Price Guide  :  (410000.0, <RecordType.ENQUIRY: 'enquiry'>)
$415,000  :  (415000.0, None)
$415,000  :  (415000.0, None)
$350 Week  :  (350.0, <RecordType.RENT: 'rent'>)
$360,000 - $395,000  :  (377500.0, None)
Private Sale  :  (None,

In [ ]:
extracted_data[5]

{'general_info': {'address': '3/27 HAROLD STREET, ASCOT VALE',
  'bed': '2',
  'bath': '1',
  'car': '1',
  'type': 'Unit/apmt'},
 'recent_price': {'date': ' June 2018', 'price': 0, 'type': '$492,000'},
 'historical_prices': [{'date': 'June 2018', 'price': 0, 'type': '$492,000'},
  {'date': 'June 2018', 'price': 0, 'type': 'Auction'},
  {'date': 'June 2018', 'price': 0, 'type': '$440,000 - $480,000'},
  {'date': 'February 2015', 'price': 0, 'type': '$455,000'},
  {'date': 'July 2012', 'price': 0, 'type': '$410,000 Price Guide'},
  {'date': 'June 2012', 'price': 0, 'type': '$415,000'},
  {'date': 'August 2012', 'price': 0, 'type': '$415,000'}]}

In [ ]:
from property_models.models import Address

Address.parse(extracted_data[5]["general_info"]["address"] + ", VIC 3032", country="australia")

NotImplementedError: Cannot parse address for 'australia'